<a href="https://colab.research.google.com/github/LuqLu/ml-proj/blob/main/YOLOv8_car_tracking_and_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description
This app uses the `YOLOv8` model for object detection on the road.

The objects detected are: humans, cars, bicycles, motorcycles, trucks, and license plates.

It uses the `deep-sort-realtime` library for tracking detected cars, bicycles, motorcycles, and trucks in order to count what number of them pass through designated areas on roads.

Additionally, it uses a blurring mechanism to anonymize license plates and people detected within the images.

To detect vehicles and people, the `YOLOv8m` model was used, which can detect these objects without additional training.

The `YOLOv8m` model was also used for license plate detection, but this time it was additionally trained on a set of `1500` license plate images.

The performance of the algorithm has been verified not with video showing the road from above, but with car camera video, which makes vehicle detection a bit more difficult due to the smaller angle at which vehicles are visible.

![picture](https://github.com/LuqLu/ml-proj/blob/main/car_counting.gif?raw=true)

### Instalation of required packages

The `ultralytics` package provides a `YOLO` model which is a high-speed, high-accuracy object detection and image segmentation model.

The `deep-sort-realtime` package provides an implementation of `Deep SORT` algorithm that combines a deep learning-based object detector with a `SORT` (Simple Online Realtime Tracking) algorithm.

In [ ]:
%pip install ultralytics
%pip install deep-sort-realtime

### Import of necessary Python libraries and modules

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import numpy as np
import os
import IPython
import datetime

from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from moviepy.video.io import ffmpeg_tools
from IPython.display import Video, display
from tqdm.notebook import tqdm
from collections import deque
from dataclasses import dataclass

### The Config class stores application parameters for fixed values.
The `YOLOv8l` model was used for vehicle detection. For vehicle detection we use original `YOLO` model which is pretrained on the `COCO` dataset. `COCO` dataset contains many classes, including `bicycle`, `car`, `motorbike`, `bus`, `truck` therefore this model should correctly detect mentioned objects.

The `YOLOv8m` model was used for license plate recognition, which was further trained on a set of `1500` license plate images. To use the trained model (the weights of the model that achieved the best performance on a validation set) we refer to the file `best.pt` in the `weights` directory.

In [ ]:
@dataclass(frozen=True)
class Config:
  """
  Configuration class for storing application settings and parameters.
  """

  # Path to the main YOLO model file
  MODEL = '/content/yolov8m.pt'
  # Path to the license plate detection model file
  LICENCE_PLATE_MODEL = '/content/drive/MyDrive/yolo/runs/detect/yolov8_licence_plate/weights/best.pt'
  # Coordinates of the starting point of line A
  START_LINE_A = (10, 1000)
  # Coordinates of the ending point of line A
  END_LINE_A = (1000, 1000)
  # Coordinates of the starting point of line B
  START_LINE_B = (1500, 1000)
  # Coordinates of the ending point of line B
  END_LINE_B = (2360, 1000)
  # Coordinates of the starting point of line C
  START_LINE_C = (2050, 950)
  # Coordinates of the ending point of line C
  END_LINE_C = (2400, 950)
  # List of class IDs to detect with YOLO model
  CLASS_IDS_TO_DETECT = [0, 1, 2, 3, 5, 7]
  # Size of the image on which the model will perform the prediction
  IMGSZ = (1600, 2560)
  # Kernel size for blurring images (Odd number required)
  KSIZE = (35, 35)
  # Maximum age for objects tracked by the DeepSort tracker
  MAX_TRACKER_AGE = 50
  # Maximum length of each deque storing tracking points
  MAX_DEQUE_LENGHT = 32
  # Maximum number of deques used for tracking
  MAX_DEQUE_COUNT = 1000

### In the VideoTools class we create methods useful for operating on video files
Methods:
- `get_video` - open a video file specified by the given path
- `get_extracted_video` - extracts a portion of a video file and returns a VideoCapture object for the extracted video
- `get_video_params` - retrieve such parameters of a video like: frame height and width, fps, frame count
- `display_video_info` - display information about the given video
- `create_video_writer` - create a VideoWriter object to write processed video frames

In [ ]:
class VideoTools:
  """
  A utility class for working with video files.

  This class provides static methods to perform various operations
  on video files, such as loading videos and extracting frames.
  """

  @staticmethod
  def get_video(path):
    """
    Open a video file specified by the given path.

    Args:
        path (str): Path to the video file.

    Returns:
        cv2.VideoCapture: VideoCapture object for passed video.

    Raises:
        ValueError: If the video file cannot be opened.
    """
    file = cv2.VideoCapture(path)
    if not file.isOpened():
      raise ValueError("Error: Video file cannot be opened.")
    return file

  @staticmethod
  def get_extracted_video(path, start_time, end_time, output_path=None):
    """
    Extracts a portion of a video file and returns a VideoCapture object for the extracted video.

    Args:
        path (str): Path to the input video file.
        start_time (float): Start time of the portion to extract (in seconds).
        end_time (float): End time of the portion to extract (in seconds).
        output_path (str, optional): Path to save the extracted video. If not specified, a default path is used.

    Returns:
        cv2.VideoCapture: VideoCapture object for the extracted video.

    Raises:
        ValueError: If the extracted video file cannot be opened.
    """
    if output_path is None:
      output_path = '/content/extracted_video.mp4'

    # Extract a portion of the video and save it as a new file
    ffmpeg_tools.ffmpeg_extract_subclip(path, start_time, end_time, output_path)

    # Open the extracted video file
    file = cv2.VideoCapture(output_path)
    if not file.isOpened():
      raise ValueError("Error: Failed to open extracted video file.")

    # Delete the extracted video file to avoid cluttering the file system
    os.remove(output_path)

    return file

  @staticmethod
  def get_video_params(video):
    """
    Retrieve various parameters of a video.

    Args:
        video (cv2.VideoCapture): VideoCapture object representing the video.

    Returns:
        tuple: A tuple containing the height, width, frames per second (fps), and total frame count of the video.
    """
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    return height, width, fps, frame_count

  @staticmethod
  def display_video_info(video):
    """
    Display information about the given video.

    Args:
        video (cv2.VideoCapture): VideoCapture object representing the video.
    """
    height, width, fps, frame_count = VideoTools.get_video_params(video)
    seconds = round(frame_count / fps)
    video_time = datetime.timedelta(seconds=seconds)
    print(f'Original video dim: {(height, width)}')
    print(f'Video fps: {fps}')
    print(f'Number of frames: {frame_count}')
    print(f'Duration in seconds: {seconds}')
    print(f'Video time: {video_time}')

  @staticmethod
  def create_video_writer(video, output_file_path=None):
    """
    Create a VideoWriter object to write processed video frames.

    Args:
        video (cv2.VideoCapture): VideoCapture object representing the video.
        output_file_path (str): The path to save the processed video.

    Returns:
        cv2.VideoWriter: A VideoWriter object for writing video frames.
    """
    if output_file_path is None:
      output_file_path = '/content/output_video.mp4'

    height, width, fps, frame_count = VideoTools.get_video_params(video)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    image_size = (width, height)
    return cv2.VideoWriter(output_file_path, fourcc, fps, image_size)

### The GeometricUtils class contains a method for calculating midpoint for given two points.
Methods:
- `calculate_midpoint` - calculate the midpoint between two points

We will use this method to determine where in the line we will put the number of vehicles that have passed the line.

In [ ]:
class GeometricUtils:
  """
  A utility class for performing geometric calculations.
  """

  @staticmethod
  def calculate_midpoint(a_point, b_point):
    """
    Calculate the midpoint between two points.

    Args:
        a_point (tuple): Coordinates of the first point (x1, y1).
        b_point (tuple): Coordinates of the second point (x2, y2).

    Returns:
        tuple: Coordinates of the midpoint (xc, yc).

    Raises:
        ValueError: If the input format is invalid.
    """
    try:
      x1, y1 = a_point
      x2, y2 = b_point
      xc = (x1 + x2) // 2
      yc = (y1 + y2) // 2
      return xc, yc
    except (TypeError, ValueError):
      raise ValueError('Invalid input format. Expected tuples with two elements.')

### In the ImageProcessor class we create methods useful for operating on images (single frames of a video file)
Methods:
- `blur_picture` - blurs regions of an image based on detections (we want to blur license plates and images of people in the processed video)
- `draw_bbox` - draw a bounding box on an image along with class name and track id
- `add_description` - add descriptions onto an image

In [ ]:
class ImageProcessor:
  """
  A utility class for image processing operations.

  This class provides static methods for performing various image processing tasks,
  including blurring, calculating midpoints, and adding descriptions to images.
  """

  @staticmethod
  def blur_picture(img, detections):
    """
    Blurs regions of an image based on detections.

    Args:
        img (numpy.ndarray): Input image.
        detections (list): List of dictionaries representing detected objects.
            Each dictionary should contain a 'box' key representing the bounding box coordinates.

    Returns:
        numpy.ndarray: Image with blurred regions.
    """
    for obj in detections:
      x1,y1,x2,y2 = obj['box']
      x1, y1, x2, y2 = round(x1), round(y1), round(x2), round(y2)
      roi = img[y1:y2, x1:x2]
      blurred_roi = cv2.GaussianBlur(roi, Config.KSIZE, 0)
      img[y1:y2, x1:x2] = blurred_roi

    return img

  @staticmethod
  def draw_lines(img):
    """
    Draws three lines on the given image and overlays it with a copy of the image.

    Args:
        img (numpy.ndarray): The image on which the lines will be drawn.

    Returns:
        numpy.ndarray: The modified image with lines drawn and overlaid.
    """
    # Create a copy of the image to use as an overlay
    overlay = img.copy()

    # Draw the lines using predefined start and end points
    cv2.line(img, Config.START_LINE_A, Config.END_LINE_A, (0, 255, 0), 15)
    cv2.line(img, Config.START_LINE_B, Config.END_LINE_B, (255, 0, 0), 15)
    cv2.line(img, Config.START_LINE_C, Config.END_LINE_C, (0, 0, 255), 15)

    # Overlay the original image with the copy using addWeighted to blend them
    img = cv2.addWeighted(overlay, 0.5, img, 0.5, 0)

    return img

  @staticmethod
  def draw_bbox(img, model, bbox, track_id, class_id):
    """
    Draw a bounding box on an image along with class name and track ID.

    Args:
        img: The image on which to draw the bounding box.
        model: Model object capable of making predictions.
        bbox: A list containing the coordinates of the bounding box in the format [x1, y1, x2, y2].
        track_id: The identifier of the tracked object.
        class_id: The identifier of the predicted class.

    Returns:
        img: The image with the bounding box, class name, and track id drawn on it.

    Raises:
        ValueError: If class_id is out of range.
    """
    if img is None or bbox is None or len(bbox) != 4:
      raise ValueError('Invalid input: img and bbox must be provided and bbox must be of length 4.')
    x1, y1, x2, y2 = map(int, bbox)

    # Get class name
    class_names = model.model.names
    if class_id < 0 or class_id >= len(class_names):
      raise ValueError('Invalid class_id: class_id out of range.')
    class_name = class_names[class_id]

    # Create a list of random colors to represent each class
    np.random.seed(10)  # To get the same colors
    colors = np.random.randint(0, 255, size=(len(class_names), 3))

    # Get the color associated with the class name
    color = tuple(map(int, colors[class_id]))

    # Draw bounding box
    cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)

    # Draw class name and track id
    text = str(track_id) + " - " + class_name
    cv2.rectangle(img, (x1 - 1, y1 - 20), (x1 + len(text) * 12, y1), color, -1)
    cv2.putText(img, text, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    return img

  @staticmethod
  def add_description(img, model, all_counters, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1, color=(255, 255, 255), thickness=2):
    """
    Add descriptions and counters onto an image.

    Args:
        img (numpy.ndarray): Input image.
        model: Model object capable of making predictions.
        all_counters (dict): A dictionary containing counts of the total number of vehicles and individual types of vehicles
            for each of the zones under consideration
        font (int): Font type. Default is cv2.FONT_HERSHEY_SIMPLEX.
        font_scale (float): Font scale factor. Default is 0.5.
        color (tuple): Text color specified as a tuple of three integers representing BGR values. Default is (255, 255, 255) (white).
        thickness (int): Thickness of the text. Default is 2.

    Returns:
        numpy.ndarray: Image with descriptions and counters added.
    """
    texts = ['A', 'B', 'C']
    text_positions = [Config.START_LINE_A, Config.START_LINE_B, Config.START_LINE_C]
    counters = [all_counters['counter_A'], all_counters['counter_B'], all_counters['counter_C']]
    counter_positions = [
        GeometricUtils.calculate_midpoint(Config.START_LINE_A, Config.END_LINE_A),
        GeometricUtils.calculate_midpoint(Config.START_LINE_B, Config.END_LINE_B),
        GeometricUtils.calculate_midpoint(Config.START_LINE_C, Config.END_LINE_C)
    ]

    # Draw the total number of vehicles passing the lines
    for text, text_position, counter, counter_position in zip(texts, text_positions, counters, counter_positions):
      cv2.putText(img, text, text_position, font, font_scale, color, thickness)
      cv2.putText(img, str(counter), counter_position, font, font_scale, color, thickness)

    # Summary of the type of vehicle and the number of its occurrences
    dict_classes = model.model.names
    result_A = [f'{dict_classes[k]}: {i}' for k, i in all_counters['vehicle_counter_A'].items()]
    result_B = [f'{dict_classes[k]}: {i}' for k, i in all_counters['vehicle_counter_B'].items()]
    result_C = [f'{dict_classes[k]}: {i}' for k, i in all_counters['vehicle_counter_C'].items()]

    # Draw the counting of type of vehicles
    font_color = (255, 255, 0)
    margine = 70 # distance from the top edge of the image

    cv2.putText(img, 'Line A', (Config.START_LINE_A[0] + 20, margine), cv2.FONT_HERSHEY_TRIPLEX, 1, font_color, 1)
    cv2.putText(img, 'Line B', (Config.START_LINE_B[0], margine), cv2.FONT_HERSHEY_TRIPLEX, 1, font_color, 1)
    cv2.putText(img, 'Line C', (Config.START_LINE_C[0] + 100, margine), cv2.FONT_HERSHEY_TRIPLEX, 1, font_color, 1)

    margine = 90
    for i in range(len(result_A)):
      margine +=30
      cv2.putText(img, f'{result_A[i]}', (Config.START_LINE_A[0] + 20, margine), cv2.FONT_HERSHEY_TRIPLEX, 1, font_color, 1)
      cv2.putText(img, f'{result_B[i]}', (Config.START_LINE_B[0], margine), cv2.FONT_HERSHEY_TRIPLEX, 1, font_color, 1)
      cv2.putText(img, f'{result_C[i]}', (Config.START_LINE_C[0] + 100, margine), cv2.FONT_HERSHEY_TRIPLEX, 1, font_color, 1)

    return img

### The main class that performs detection and tracking of the objects considered above. In addition to its own methods, it uses methods from the above classes: VideoTools, ImageProcessor.
Methods:
- `load_model` - load a model from a given path
- `predict` - predict the classes of objects in the given image using the provided model
- `get_bbox_params` - extract bounding box parameters (coordinates, class labels, confidences) from the results
- `convert_to_tracker_format` - convert detections to the format expected by the tracker
- `count_vehicles` - counts vehicles based on the bounding box and track id
- `track_detect` - perform object tracking on the given image using the provided detections and tracker

In [ ]:
class ObjectDetection:
  """
  A class for performing object detection and tracking in videos.

  This class encapsulates functionality for loading object detection models,
  predicting object bounding boxes in video frames, converting detections to
  tracker-compatible format, counting detected objects, and tracking objects
  across frames in a video stream.

  Attributes:
      video: The input video file or video stream for object detection and tracking.
  """

  def __init__(self, video):
    """
    Initialize ObjectDetection object.

    Args:
        video: Video object.
    """
    self.video = video
    self.model = self.load_model(Config.MODEL)
    self.licence_plate_model = self.load_model(Config.LICENCE_PLATE_MODEL)
    self.points = [deque(maxlen=Config.MAX_DEQUE_LENGHT) for _ in range(Config.MAX_DEQUE_COUNT)]
    self.counter_A = 0
    self.counter_B = 0
    self.counter_C = 0
    self.vehicle_counter_A = dict.fromkeys(Config.CLASS_IDS_TO_DETECT[1:], 0)
    self.vehicle_counter_B = dict.fromkeys(Config.CLASS_IDS_TO_DETECT[1:], 0)
    self.vehicle_counter_C = dict.fromkeys(Config.CLASS_IDS_TO_DETECT[1:], 0)

    self.tracked_ids = set()

  def load_model(self, path):
    """
    Load a model from the given path.

    Args:
        path (str): Path to the model file.

    Returns:
        model: The loaded model object.
    """
    model = YOLO(path)
    return model

  def predict(self, img, model, classes, verbose=False):
    """
    Predict the classes of objects in the given image using the provided model.

    Args:
        img: Input image for prediction.
        model: Model object capable of making predictions.
        classes (list): List of classes to predict by the model.
        verbose (bool, optional): Verbosity mode. Defaults to False.

    Returns:
        results: Results of the prediction.
    """
    results = model.predict(img, imgsz=Config.IMGSZ, classes=classes, device=0, verbose=verbose)
    return results

  def get_bbox_params(self, results):
    """
    Extract bounding box parameters (coordinates, class labels, confidences) from the results.

    Args:
        results: The results of object detection.

    Returns:
        detections: A list of dictionaries containing bounding box parameters.
            Each dictionary contains keys 'box', 'class', and 'conf'.
    """
    boxes = results[0].boxes.xyxy.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()

    detections = []
    for box, cls, conf in zip(boxes, classes, confs):
      detection = {
          'box': box,
          'class': cls,
          'conf': conf
      }
      detections.append(detection)
    return detections

  def convert_to_tracker_format(self, detections):
    """
    Convert detections to the format expected by the tracker.

    Args:
        detections (list): A list of dictionaries containing detection information.

    Returns:
        list: A list of detections in the format [[xmin, ymin, w, h], confidence, class_id].
    """
    results = []
    for obj in detections:
      xmin, ymin, xmax, ymax = obj['box']
      confidence = obj['conf']
      class_id = int(obj['class'])
      bbox_width = xmax - xmin
      bbox_height = ymax - ymin
      results.append([[xmin, ymin, bbox_width, bbox_height], confidence, class_id])
    return results

  def count_vehicles(self, img, bbox, track_id, class_id):
    """
    Counts vehicles based on the bounding box and track id.

    Args:
        img (numpy.ndarray): Input image.
        bbox (tuple): Bounding box coordinates (x1, y1, x2, y2).
        track_id (int): Unique identifier for the object track.
        class_id (int): Identifier for the object class.

    Returns:
        numpy.ndarray: Image with added centres of bounding boxes.
    """
    x1, y1, x2, y2 = map(int, bbox)
    # Determination of the interior point of bounding box
    interior_x = int((x1 + x2) / 2)
    center_y = int((y1 + y2) / 2)
    if class_id == 1:
      interior_y = y2
    else:
      interior_y = int((center_y + y2) / 2)

    # In general, you can set any percentage of the bounding box height.
    # interior_y = (y2 - y1) * 0.75 + y1

    # Append the interior point of the current object to the points list
    self.points[track_id].append((interior_x, interior_y))

    # Get the last point from the points list and draw it
    last_point_x = self.points[track_id][0][0]
    last_point_y = self.points[track_id][0][1]

    # Count the number of vehicles passing the lines
    # Check the condition whether the object has crossed the line
    if interior_y > Config.START_LINE_A[1] and Config.START_LINE_A[0] < interior_x < Config.END_LINE_A[0] and last_point_y < Config.START_LINE_A[1]:
      self.counter_A += 1
      self.vehicle_counter_A[class_id] += 1
      self.points[track_id].clear()
    elif interior_y < Config.START_LINE_B[1] and Config.START_LINE_B[0] < interior_x < Config.END_LINE_B[0] and last_point_y > Config.START_LINE_B[1]:
      self.counter_B += 1
      self.vehicle_counter_B[class_id] += 1
      self.points[track_id].clear()
    elif interior_y < Config.START_LINE_C[1] and Config.START_LINE_C[0] < interior_x < Config.END_LINE_C[0] and last_point_y > Config.START_LINE_C[1]:
      self.counter_C += 1
      self.vehicle_counter_C[class_id] += 1
      self.points[track_id].clear()
    return img

  def track_detect(self, img, detections, tracker):
    """
    Perform object tracking on the given image using the provided detections and tracker.

    Args:
        img: The input image for object tracking.
        detections (list): A list of detections in the format [[xmin, ymin, w, h], confidence, class_id].
        tracker: The object tracker used for tracking.

    Returns:
        img: The input image with object tracking results.
    """
    # Update tracker with the new detections
    tracks = tracker.update_tracks(detections, frame=img)

    # Loop over the tracks
    for track in tracks:
      if not track.is_confirmed() or track.time_since_update > 1 or track.det_class == 0:
        continue
      # Get the bounding box of the object, the name of the object, and the track id
      bbox = track.to_tlbr()
      track_id = int(track.track_id)
      class_id = track.det_class

      # Draw the bounding box of the object and the track id
      img = ImageProcessor.draw_bbox(img, self.model, bbox, track_id, class_id)

      img = self.count_vehicles(img, bbox, track_id, class_id)

    counters = {
        'counter_A': self.counter_A,
        'counter_B': self.counter_B,
        'counter_C': self.counter_C,
        'vehicle_counter_A': self.vehicle_counter_A,
        'vehicle_counter_B': self.vehicle_counter_B,
        'vehicle_counter_C': self.vehicle_counter_C
    }

    img = ImageProcessor.add_description(img, self.model, counters)

    return img


  def __call__(self):
    """
    Process the video by reading frames, making predictions, processing frames, tracking objects, and saving the output video.

    Returns:
        None
    """
    tracker = DeepSort(max_age=Config.MAX_TRACKER_AGE)
    output_video = VideoTools.create_video_writer(self.video)

    frame_count = int(self.video.get(cv2.CAP_PROP_FRAME_COUNT))
    for _ in tqdm(range(frame_count)):
      success, frame = self.video.read()
      assert success, 'Failed to read a frame'

      # Draw lines designating vehicle counting locations
      frame = ImageProcessor.draw_lines(frame)

      # Detect and blur licence plates in the frame
      licence_plate_results = self.predict(frame, self.licence_plate_model, None, False)
      licence_plate_boxes = self.get_bbox_params(licence_plate_results)
      frame = ImageProcessor.blur_picture(frame, licence_plate_boxes)

      # Detect other object
      results = self.predict(frame, self.model, Config.CLASS_IDS_TO_DETECT, False)
      boxes = self.get_bbox_params(results)

      # Blur person in the frame
      person_boxes = [box for box in boxes if box['class'] == 0]
      frame = ImageProcessor.blur_picture(frame, person_boxes)

      tracker_boxes = self.convert_to_tracker_format(boxes)
      frame = self.track_detect(frame, tracker_boxes, tracker)

      # Save transformed frames in an output video
      output_video.write(frame)

    # Release resources
    output_video.release()
    cv2.destroyAllWindows()

### Launching the application for the given video file

In [ ]:
video_path = 'path_to_video_file'
video = VideoTools.get_video(video_path)
# Or you can use just a portion of the video
# video = VideoTools.get_extracted_video(video_path, 90, 120)
VideoTools.display_video_info(video)
detection = ObjectDetection(video)
detection()

In [ ]:
video_path = '/content/drive/MyDrive/yolo/20230521150750_000050.MP4'

video = VideoTools.get_extracted_video(video_path, 94, 123)
#video = VideoTools.get_video(video_path) # Or you can use the whole video
VideoTools.display_video_info(video)
detection = ObjectDetection(video)
detection()

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Original video dim: (1600, 2560)
Video fps: 30.0
Number of frames: 870
Duration in seconds: 29
Video time: 0:00:29


100%|██████████| 49.7M/49.7M [00:00<00:00, 244MB/s]


  0%|          | 0/870 [00:00<?, ?it/s]